In [6]:
from dotenv import load_dotenv


load_dotenv()

True

In [7]:
import os, sys
from pathlib import Path


def find_src_folder():
    current = Path(os.getcwd()).resolve()
    for p in [current] + list(current.parents):
        src = p / "src"
        if src.exists():
            return src
    raise RuntimeError("src 폴더를 찾을 수 없습니다.")


src_path = find_src_folder()
sys.path.append(str(src_path))

## Super Dungeon Agent 테스트 (Monster Agent 통합)

In [8]:
from agents.dungeon.super.dungeon_agent import create_super_dungeon_graph
import json

# Super Dungeon Graph 생성
super_graph = create_super_dungeon_graph()
print("✅ Super Dungeon Graph 생성 완료")

✅ Super Dungeon Graph 생성 완료


### 테스트 실행

In [9]:
from agents.dungeon.monster.monster_database import MONSTER_DATABASE

# 테스트 State 구성
test_state = {
    "dungeon_base_data": {
        "dungeon_id": 1,
        "floor_count": 3,
        "rooms": [
            {"room_id": 1, "room_type": "start", "position": {"x": 0.0, "y": 0.0}},
            {"room_id": 2, "room_type": "monster", "position": {"x": 0.2, "y": 0.1}},
            {"room_id": 3, "room_type": "monster", "position": {"x": 0.4, "y": 0.2}},
            {"room_id": 4, "room_type": "treasure", "position": {"x": 0.6, "y": 0.3}},
            {"room_id": 5, "room_type": "monster", "position": {"x": 0.8, "y": 0.4}},
            {"room_id": 6, "room_type": "boss", "position": {"x": 0.9, "y": 0.5}},
        ],
    },
    "heroine_data": {
        "heroine_id": 1,
        "name": "레티아",
        "event_room": 3,
        "memory_progress": 50,
    },
    "heroine_stat": {
        "hp": 500,
        "strength": 20,
        "dexterity": 15,
        "intelligence": 10,
        "defense": 10,
        "critChance": 15.0,
        "attackSpeed": 1.5,
        "moveSpeed": 400,
        "skillDamageMultiplier": 1.2,
    },
    "heroine_memories": [],
    "monster_db": MONSTER_DATABASE,
    "dungeon_player_data": {
        "affection": 50,
        "sanity": 80,
        "difficulty_level": "normal",
    },
    "used_events": [],
    "event_result": {},
    "filled_dungeon_data": {},
    "difficulty_log": {},
    "final_dungeon_json": {},
}

print("=" * 70)
print("Super Dungeon Agent 테스트 시작")
print("=" * 70)

# Graph 실행
result = super_graph.invoke(test_state)

print("\n" + "=" * 70)
print("✅ 실행 완료")
print("=" * 70)

Super Dungeon Agent 테스트 시작

[Event Node] 이벤트 생성 시작...

[Monster Node] 몬스터 밸런싱 시작...
[calculate_combat_score_node] 플레이어 전투력: 194.00
[calculate_combat_score_node] HP: 500, STR: 20, DEX: 15
[llm_strategy_node DEBUG] combat_score: 194.0
[llm_strategy_node DEBUG] heroine_stat type: <class 'dict'>
[llm_strategy_node DEBUG] heroine_stat: {'hp': 500, 'strength': 20, 'dexterity': 15, 'intelligence': 10, 'defense': 10, 'critChance': 15.0, 'attackSpeed': 1.5, 'moveSpeed': 400, 'skillDamageMultiplier': 1.2}
[llm_strategy_node DEBUG] dungeon_player_data: {'affection': 50, 'sanity': 80, 'difficulty_level': 'normal'}
[llm_strategy_node DEBUG] hero_summary:

전투 스탯:
- HP: 500
- 근력(Strength): 20
- 기량(Dexterity): 15
- 치명타 확률: 15.0%
- 공격 속도: 1.50x
- 스킬 데미지 배율: 1.20x
- 종합 전투력: 194.0 (플레이어)

던전 진행 상황:
- 현재 층: 3
- 난이도: 1
- 호감도: 50
- 정신력: 80

[heroine_memories_node] 히로인 ID: 1
[heroine_memories_node] 기억 진척도: 50
[heroine_memories_node] 해금된 기억 개수: 2
[selected_main_event_node] 선택된 이벤트: 떨쳐내기 힘든 유혹
[selected_main_e

### 결과 확인

In [5]:
final_json = result.get("final_dungeon_json", {})

print("\n📊 최종 결과:")
print(json.dumps(final_json, indent=2, ensure_ascii=False))
print("\n\n🎯 요약:")
print(f"Event - Main: {final_json.get('events', {}).get('main_event', 'N/A')[:50]}...")
print(f"Monster - 총 {final_json.get('monster_stats', {}).get('total_count', 0)}마리")
print(f"  - 보스: {final_json.get('monster_stats', {}).get('boss_count', 0)}마리")
print(f"  - 일반: {final_json.get('monster_stats', {}).get('normal_count', 0)}마리")
print(
    f"Difficulty - x{final_json.get('difficulty_info', {}).get('ai_multiplier', 0):.2f}"
)
print(
    f"Combat Score - {final_json.get('difficulty_info', {}).get('combat_score', 0):.2f}"
)

print("\n\n🏰 방별 몬스터 배치 (언리얼 엔진용):")
for room in final_json.get("dungeon_data", {}).get("rooms", []):
    room_type = room.get("room_type")
    monsters = room.get("monsters", [])
    if monsters:
        print(f"\n방 #{room['room_id']} ({room_type}):")
        for m in monsters:
            position = m.get("position", {})
            print(
                f"  - 몬스터 ID: {m['monster_id']}, 이름: {m['monster_name']}, 타입: {m['monster_type']}, 위협도: {m['threat_level']:.2f}"
            )
            print(
                f"    좌표: x={position.get('x', 0):.2f}, y={position.get('y', 0):.2f}, z={position.get('z', 0):.2f}"
            )


📊 최종 결과:
{
  "dungeon_data": {
    "dungeon_id": 1,
    "floor_count": 3,
    "rooms": [
      {
        "room_id": 1,
        "room_type": "start",
        "position": {
          "x": 0.0,
          "y": 0.0
        }
      },
      {
        "room_id": 2,
        "room_type": "monster",
        "position": {
          "x": 0.2,
          "y": 0.1
        }
      },
      {
        "room_id": 3,
        "room_type": "monster",
        "position": {
          "x": 0.4,
          "y": 0.2
        }
      },
      {
        "room_id": 4,
        "room_type": "treasure",
        "position": {
          "x": 0.6,
          "y": 0.3
        }
      },
      {
        "room_id": 5,
        "room_type": "monster",
        "position": {
          "x": 0.8,
          "y": 0.4
        }
      },
      {
        "room_id": 6,
        "room_type": "boss",
        "position": {
          "x": 0.9,
          "y": 0.5
        }
      }
    ]
  },
  "events": {
    "main_event": "[조여오는 죄책감]\nEvent C